In [18]:
import pandas as pd

import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

from antipark import *
from antipark.models import Product, Category

# Save Db to Excel

In [20]:
products = Product.query.all()

2016-09-15 11:39:03,190 INFO sqlalchemy.engine.base.Engine SELECT products.id AS products_id, products.category AS products_category, products.subcategory AS products_subcategory, products.title AS products_title, products.specs AS products_specs, products.price AS products_price, products.stage_price AS products_stage_price, products.image AS products_image 
FROM products
2016-09-15 11:39:03,192 INFO sqlalchemy.engine.base.Engine {}


In [30]:
columns = [column for column in dir(products[0]) if not (column.startswith('_') or
                                                         column.startswith('query') or
                                                         column.startswith('meta'))]
columns

['category',
 'id',
 'image',
 'price',
 'specs',
 'stage_price',
 'subcategory',
 'title']

In [31]:
price_cols = ['id', 'title', 'price', 'stage_price']

In [37]:
from collections import defaultdict

prods = defaultdict(list)
for col in price_cols:
    for product in products:
        prods[col].append(getattr(product, col))    

In [40]:
df = pd.DataFrame(prods)
df.set_index('id', inplace=True)
df.head(5)

,price,stage_price,title
id,,,
1027,899,1199,Столбик парковочный бетонируемый
1028,999,399,Столбик парковочный анкерный
1029,1999,-,Столбик парковочный передвижной
1030,1999,1199,Столбик съемный с бетонируемой закладной
1031,1999,1199,Столбик парковочный складной


In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 329 entries, 1027 to 1355
Data columns (total 3 columns):
price          329 non-null object
stage_price    329 non-null object
title          329 non-null object
dtypes: object(3)
memory usage: 10.3+ KB


In [45]:
df.to_excel('cur_base.xlsx')

# Read Excel and Update Db

In [110]:
df_new = pd.read_excel('cur_base.xlsx', index_col='id')
df_new.head(5)

,price,stage_price,title
id,,,
1027,899,1199,Столбик парковочный бетонируемый
1028,999,399,Столбик парковочный анкерный
1029,1999,-,Столбик парковочный передвижной
1030,1999,1199,Столбик съемный с бетонируемой закладной
1031,1999,1199,Столбик парковочный складной


In [111]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 329 entries, 1027 to 1355
Data columns (total 3 columns):
price          329 non-null object
stage_price    329 non-null object
title          329 non-null object
dtypes: object(3)
memory usage: 10.3+ KB


In [105]:
# if smth was wrong
db.session.rollback()

In [ ]:
for prod in df_new.itertuples():
    upd_product = Product.query.get(prod.Index)
    for field in prod._fields[1:]:
        setattr(upd_product, field, getattr(prod, field))
    db.session.commit()

2016-09-15 12:25:40,243 INFO sqlalchemy.engine.base.Engine SELECT products.id AS products_id, products.category AS products_category, products.subcategory AS products_subcategory, products.title AS products_title, products.specs AS products_specs, products.price AS products_price, products.stage_price AS products_stage_price, products.image AS products_image 
FROM products 
WHERE products.id = %(param_1)s
2016-09-15 12:25:40,245 INFO sqlalchemy.engine.base.Engine {'param_1': 1027}
2016-09-15 12:25:40,249 INFO sqlalchemy.engine.base.Engine COMMIT
2016-09-15 12:25:40,257 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2016-09-15 12:25:40,259 INFO sqlalchemy.engine.base.Engine SELECT products.id AS products_id, products.category AS products_category, products.subcategory AS products_subcategory, products.title AS products_title, products.specs AS products_specs, products.price AS products_price, products.stage_price AS products_stage_price, products.image AS products_image 
FROM produ